# Produce Messages in AVRO

In [ ]:
import env
# Import required libraries
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer

# Configure the Kafka producer
conf = {
    "bootstrap.servers": env.KAFKA_SERVICE_URI,
    "security.protocol": "SSL",
    "ssl.ca.location": env.CERTIFICATES_FOLDER + "/ca.pem",
    "ssl.certificate.location": env.CERTIFICATES_FOLDER + "/service.cert",
    "ssl.key.location": env.CERTIFICATES_FOLDER + "/service.key",
    "client.id": "0001",
    "acks": 1
}

# Configure the Avro schema registry
schema_registry_conf = {
    "url": "https://{}:{}@{}:{}".format(env.SCHEMA_REGISTRY_USERNAME, env.SCHEMA_REGISTRY_PASSWORD, env.SCHEMA_REGISTRY_HOSTNAME, env.SCHEMA_REGISTRY_PORT)
}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

# Define the Avro schema for the message
value_str = """
{
"namespace": "example.avro",
"type": "record",
"name": "User",
"fields": [
    {"name": "name", "type": "string"},
    {"name": "age",  "type": ["int", "null"]}
]
}
"""

key_str = """
{
"namespace": "example.avro",
"type": "record",
"name": "Id",
"fields": [
    {"name": "id", "type": "int"}
]
}
"""

# Define a class for the User object
class User(object):
    def __init__(self, name, age):
        self.name = name
        self.age = age

# Define a class for the Key object
class Key(object):
    def __init__(self, id):
        self.id = id

# Converting user object to key for AvroSerializer function
def user_to_dict(user, ctx):
    return dict(
        name=user.name,
        age=user.age,
    )

# Converting key object to key for AvroSerializer function
def key_to_dict(key, ctx):
    return dict(id=key.id)

# Create the Avro serializer
avro_serializer_value = AvroSerializer(
    schema_registry_client, value_str, user_to_dict
)
avro_serializer_key = AvroSerializer(
    schema_registry_client, key_str, key_to_dict
)
# Create the string serializer for Key
string_serializer = StringSerializer("utf_8")

# Create the Kafka producer
producer = Producer(conf)

# Define the message key and payload
key = Key(id=1)
payload = User(name="John", age=30)

# Send the message to the Kafka topic
topic = "demo-topic-avro"
producer.produce(
    topic=topic,
    key=avro_serializer_key(
        key, SerializationContext(topic, MessageField.KEY)
    ),
    value=avro_serializer_value(
        payload,
        SerializationContext(topic, MessageField.VALUE),
    ),
)

# Wait for any outstanding messages to be delivered and delivery reports to be received
producer.flush()

Change the payload, this will break the consumer

In [ ]:
value_str = """
{
"namespace": "example.avro",
"type": "record",
"name": "User",
"fields": [
    {"name": "firstname", "type": "string"},
    {"name": "age",  "type": ["int", "null"]}
]
}
"""

key_str = """
{
"namespace": "example.avro",
"type": "record",
"name": "Id",
"fields": [
    {"name": "id", "type": "int"}
]
}
"""

# Define a class for the User object
class User(object):
    def __init__(self, firstname, age):
        self.firstname = firstname
        self.age = age

# Define a class for the Key object
class Key(object):
    def __init__(self, id):
        self.id = id

# Converting user object to key for AvroSerializer function
def user_to_dict(user, ctx):
    return dict(
        firstname=user.firstname,
        age=user.age,
    )

# Converting key object to key for AvroSerializer function
def key_to_dict(key, ctx):
    return dict(id=key.id)

# Create the Avro serializer
avro_serializer_value = AvroSerializer(
    schema_registry_client, value_str, user_to_dict
)
avro_serializer_key = AvroSerializer(
    schema_registry_client, key_str, key_to_dict
)
# Create the string serializer for Key
string_serializer = StringSerializer("utf_8")

# Create the Kafka producer
producer = Producer(conf)

# Define the message key and payload
key = Key(id=1)
payload = User(firstname="John", age=30)

# Send the message to the Kafka topic
topic = "demo-topic-avro"
producer.produce(
    topic=topic,
    key=avro_serializer_key(
        key, SerializationContext(topic, MessageField.KEY)
    ),
    value=avro_serializer_value(
        payload,
        SerializationContext(topic, MessageField.VALUE),
    ),
)

# Wait for any outstanding messages to be delivered and delivery reports to be received
producer.flush()

In [ ]:
value_str = """
{
"namespace": "example.avro",
"type": "record",
"name": "User",
"fields": [
    {"name": "name", "type": "string"},
    {"name": "age",  "type": ["int", "null"]},
    {"name": "nationality", "type": "string", "default":"italian"}
]
}
"""

key_str = """
{
"namespace": "example.avro",
"type": "record",
"name": "Id",
"fields": [
    {"name": "id", "type": "int"}
]
}
"""

# Define a class for the User object
class User(object):
    def __init__(self, name, age, nationality):
        self.name = name
        self.age = age
        self.nationality = nationality

# Define a class for the Key object
class Key(object):
    def __init__(self, id):
        self.id = id

# Converting user object to key for AvroSerializer function
def user_to_dict(user, ctx):
    return dict(
        name=user.name,
        age=user.age,
        nationality=user.nationality
    )

# Converting key object to key for AvroSerializer function
def key_to_dict(key, ctx):
    return dict(id=key.id)

# Create the Avro serializer
avro_serializer_value = AvroSerializer(
    schema_registry_client, value_str, user_to_dict
)
avro_serializer_key = AvroSerializer(
    schema_registry_client, key_str, key_to_dict
)
# Create the string serializer for Key
string_serializer = StringSerializer("utf_8")

# Create the Kafka producer
producer = Producer(conf)

# Define the message key and payload
key = Key(id=1)
payload = User(name="Francesco", age=25, nationality="Italian")

# Send the message to the Kafka topic
topic = "demo-topic-avro"
producer.produce(
    topic=topic,
    key=avro_serializer_key(
        key, SerializationContext(topic, MessageField.KEY)
    ),
    value=avro_serializer_value(
        payload,
        SerializationContext(topic, MessageField.VALUE),
    ),
)

# Wait for any outstanding messages to be delivered and delivery reports to be received
producer.flush()